# Project 1: Machine Learning

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from implementations import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper import load_csv_data
from processing import clean_data, standardize

# Import of the data
Import of the train and test data

In [2]:
y_train, x_train, id_train = load_csv_data("../data/train.csv")
print(x_train.shape, y_train.shape)

(250000, 30) (250000,)


In [ ]:
var_names = np.genfromtxt("../data/train.csv",
              delimiter=',',
              encoding='UTF-8-sig',
              dtype=None,
              names=True).dtype.names[2:]

In [14]:
y_test, x_test, id_test = load_csv_data("../data/test.csv")
print(x_test.shape, y_test.shape)

(568238, 30) (568238,)


# Preprocess data
We pre process the data to get a clean dataset

In [3]:
x_train_cleaned, _, _ = standardize(clean_data(x_train))
x_train_cleaned

array([[ 1.07398883,  0.09429484,  0.61533879, ..., -0.47463072,
        -0.51655392,  0.79217235],
       [ 1.32752548,  0.28741243,  0.67636725, ..., -0.48873678,
        -0.48862393,  0.03302941],
       [ 0.77990572,  1.34146227,  0.9327364 , ..., -0.48873678,
        -0.48862393,  0.01074183],
       ...,
       [ 0.70144218,  0.19440272,  0.36720758, ..., -0.48873678,
        -0.48862393, -0.01475064],
       [ 0.58288357, -0.27012672,  0.28790896, ..., -0.48873678,
        -0.48862393, -0.48862393],
       [ 0.77990572,  0.3324164 ,  0.31069307, ..., -0.48873678,
        -0.48862393, -0.48862393]])

# Learning algorithms

### Least squares

In [4]:
weight, loss = least_squares(y_train, x_train_cleaned)
print(compute_mse_loss(y_train, x_train_cleaned, weight))

0.509968


### Least squares with ridges regression

In [6]:
weight, loss = ridge_regression(y_train, x_train_cleaned, 10)
print(loss)

0.509968


### Least squares with gradient descent

In [7]:
weight, loss = mean_squared_error_gd(y_train, x_train_cleaned, np.ones((30,)), 100, 1e-4)
loss

(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30) (250000,)
(250000, 30

1.24932

### Least squares with stochastic gradient descent

In [65]:
weight, loss = mean_squared_error_sgd(y_train, x_train_cleaned, np.ones((30,)), 100, 1e-4)
loss

0.74884

### Logistic regression

In [28]:
logistic_regression(y_train, x_train_cleaned, np.ones((30,)), 100, 1e-3)

/Users/mac/Desktop/MLprojec1/scripts/implementations.py:179: RuntimeWarning: overflow encountered in exp
  loss = np.log(1 + np.exp(tx @ w)) - y * (tx @ w)
/Users/mac/Desktop/MLprojec1/scripts/implementations.py:161: RuntimeWarning: overflow encountered in exp
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/MLprojec1/scripts/implementations.py:161: RuntimeWarning: invalid value encountered in divide
  return np.exp(t) / (1 + np.exp(t))


Current iteration=0, loss=inf


(array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan]),
 nan)

In [ ]:


def logistic_regression(
    y: np.ndarray, tx: np.ndarray, initial_w: np.ndarray, max_iters: int, gamma: float
) -> list[np.ndarray, float]:
    # init parameters
    threshold = 1e-8
    losses = []
    w = initial_w

    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, gradient = calculate_nll_loss(y, tx, w), calculate_logistic_gradient(
            y, tx, w
        )
        w = w - gamma * gradient

        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return w, losses[-1]

In [30]:
x_train_cleaned

array([[ 1.38470e+02,  5.16550e+01,  9.78270e+01, ...,  1.24000e+00,
        -2.47500e+00,  1.13497e+02],
       [ 1.60937e+02,  6.87680e+01,  1.03235e+02, ..., -1.00000e-02,
        -0.00000e+00,  4.62260e+01],
       [ 1.12410e+02,  1.62172e+02,  1.25953e+02, ..., -1.00000e-02,
        -0.00000e+00,  4.42510e+01],
       ...,
       [ 1.05457e+02,  6.05260e+01,  7.58390e+01, ..., -1.00000e-02,
        -0.00000e+00,  4.19920e+01],
       [ 9.49510e+01,  1.93620e+01,  6.88120e+01, ..., -1.00000e-02,
        -0.00000e+00,  0.00000e+00],
       [ 1.12410e+02,  7.27560e+01,  7.08310e+01, ..., -1.00000e-02,
        -0.00000e+00,  0.00000e+00]])

In [ ]:

def calculate_nll_loss(y: np.ndarray, tx: np.ndarray, w: np.ndarray) -> float:
    """
    compute the cost by negative log likelihood.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a non-negative loss: float
    """
    assert y.shape[0] == tx.shape[0]
    assert tx.shape[1] == w.shape[0]

    loss = np.log(1 + np.exp(tx @ w)) - y * (tx @ w)
    return np.mean(loss)
